# IMPORTS

In [ ]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

## Setting hyperparameters and creating the file _config_

Quick explanations

In [ ]:
# Description du fichier config :
# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].

# network_features_per_convolution : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[[64,64] for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_convolution[k][1] = network_features_per_convolution[k+1][0].

# network_trainingset : string : String describing the dataset used for the training.

Name of the .json file where the network configuration will be stored

In [ ]:
filename = '/config_network.json'

## Setting the main parameters of the network

If multiple training sets are available, you can select which one you want to use here.

In [ ]:
trainingset_name = 'SEM_3classes_reduced'

Print default configuration:

In [ ]:
default_configuration()

We then set the main parameters and the architecture of the network that we want to change, compared to the default configuration above.

In [ ]:
L_struct = [{'structure':[[5,5], [3,3], [3,3], [3,3]], 'features_augmentation':'x2', 'network_first_num_features':16}]

dict_params = {'network_additional_parameters':[{'learning_rate_decay_period':120,
                                                'learning_rate_decay_rate':0.99,
                                                 'learning_rate_decay_activate':True,
                                                'batch_norm_decay_decay_period':3000,
                                                'batch_norm_decay_ending_decay':0.999,
                                                'batch_norm_decay_decay_activate':False
                                               }],
               'network_batch_norm_decay':0.9,
               'network_batch_norm':False,
               'network_trainingset':[trainingset_name],
               'network_thresholds':[[0, 0.2, 0.8]],
               'network_n_classes':3,
               'network_weighted_cost':False
              }

config_list = grid_config(L_struct, dict_params)

## Preparing the training phase

In [ ]:
path_training = '../data/'+trainingset_name+'/training/'

## Actually training the model

By default, the name is the current date. You can uncomment the second line dir_name if you want to give a specific name to the folder, but beware: it will erase the contents of the folder if it already exists !

The model data will be stored in a folder with this name, as well as the summaries that are going to be readable on tensorboard.

In [ ]:
#dir_name = time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S") 
#dir_name = 'cv_2c_nw_d5_2_b8_p10-10'
#dir_name += '_' + str(int(time.time()))[-4:]

for dir_name, config in config_list.iteritems():
    pass
path_model = os.path.join('../models/', dir_name)

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
path_model_init = None 


# Specify here the path to the initial model if needed.

if os.path.exists(path_model+filename): # if there is already a configfile
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_model+filename, 'w') as f:
        json.dump(config, f, indent=2)
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
        
        
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config_network)

print 'Chosen parameters :\n'

for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

print '\n-----------------------------------------------------------------\n'
print 'Training model ' + str(dir_name) + '...\n'

################################################### Training phase ###################################################

# Note : possible values for gpu: None, '/gpu:0'. If used withing CUDA_VISIBLE_DEVICES=?, then it will use gpu number ? by default
from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network,path_model_init=path_model_init,gpu='/gpu:0')